In [2]:
! pip install kaggle


You should consider upgrading via the 'C:\Users\marti\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [3]:
# Importing all the required modules

In [65]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
#Stopword are basically words that doesn't have meaning to your sentence 
#mean these words cannot change the sentiments of the tweets
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [7]:
#Data Processing

In [8]:
#loading data from csv file to pandas
twitter_data = pd.read_csv('Tweets.csv', encoding='ISO-8859-1')

In [9]:
#Gives the number of rows and columns
twitter_data.shape

(14640, 15)

In [10]:
#Prints first five rows
twitter_data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [11]:
#checking for null values
twitter_data.isnull().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   5462
negativereason_confidence        4118
airline                             0
airline_sentiment_gold          14600
name                                0
negativereason_gold             14608
retweet_count                       0
text                                0
tweet_coord                     13621
tweet_created                       0
tweet_location                   4733
user_timezone                    4820
dtype: int64

In [12]:
twitter_data['airline_sentiment'].value_counts()

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: count, dtype: int64

In [13]:
twitter_data['airline'].value_counts()

airline
United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: count, dtype: int64

In [14]:
#Stemming

In [15]:
#Stemming is a process of reducing a word to its root word
port_stem = PorterStemmer()

In [16]:
def stemming(content):

    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = " ".join(stemmed_content)

    return stemmed_content

In [17]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [18]:
twitter_data['stemmed_content']

0                              virginamerica dhepburn said
1               virginamerica plu ad commerci experi tacki
2        virginamerica today must mean need take anoth ...
3        virginamerica realli aggress blast obnoxi ente...
4                       virginamerica realli big bad thing
                               ...                        
14635          americanair thank got differ flight chicago
14636    americanair leav minut late flight warn commun...
14637    americanair pleas bring american airlin blackb...
14638    americanair money chang flight answer phone su...
14639    americanair ppl need know mani seat next fligh...
Name: stemmed_content, Length: 14640, dtype: object

In [21]:
twitter_data

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,stemmed_content
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),virginamerica dhepburn said
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),virginamerica plu ad commerci experi tacki
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),virginamerica today must mean need take anoth ...
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),virginamerica realli aggress blast obnoxi ente...
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),virginamerica realli big bad thing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,NaN,0.0000,American,NaN,KristenReenders,NaN,0,@AmericanAir thank you we got on a different f...,NaN,2015-02-22 12:01:01 -0800,NaN,NaN,americanair thank got differ flight chicago
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,NaN,2015-02-22 11:59:46 -0800,Texas,NaN,americanair leav minut late flight warn commun...
14637,569587242672398336,neutral,1.0000,NaN,NaN,American,NaN,sanyabun,NaN,0,@AmericanAir Please bring American Airlines to...,NaN,2015-02-22 11:59:15 -0800,"Nigeria,lagos",NaN,americanair pleas bring american airlin blackb...
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...",NaN,2015-02-22 11:59:02 -0800,New Jersey,Eastern Time (US & Canada),americanair money chang flight answer phone su...


In [25]:
twitter_data['airline'].value_counts()

airline
United            3822
US Airways        2913
American          2759
Southwest         2420
Delta             2222
Virgin America     504
Name: count, dtype: int64

In [43]:
united = []
us_airways = []
american = []
southwest = []
delta = []
virgin_america = []


for i,airline in enumerate(twitter_data['airline']):
    if airline == 'Virgin America':
        virgin_america.append(twitter_data['airline_sentiment'][i])
    elif airline == 'United':
        united.append(twitter_data['airline_sentiment'][i])
    elif airline == 'Southwest':
        southwest.append(twitter_data['airline_sentiment'][i])
    elif airline == 'Delta':
        delta.append(twitter_data['airline_sentiment'][i])
    elif airline == 'US Airways':
        us_airways.append(twitter_data['airline_sentiment'][i])
    elif airline == 'American':
        united.append(twitter_data['airline_sentiment'][i])

neutral


In [56]:
len(united)

6581

In [22]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['airline_sentiment'].values

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify = Y, random_state=2)
#20% data of twitter_data goes to X_test and Y_test. The remaining data goes to X_train and Y_train.

In [58]:
Y_train

array(['negative', 'positive', 'negative', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [70]:
#converting textual data into numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)


AttributeError: 'csr_matrix' object has no attribute 'lower'

In [73]:
print(X_test)

  (0, 8712)	0.1318951927835332
  (0, 8708)	0.3429700136227325
  (0, 8295)	0.30723505526866296
  (0, 7557)	0.2573103190964777
  (0, 6223)	0.20424312354710805
  (0, 6163)	0.3351778895289792
  (0, 5891)	0.35052865577962433
  (0, 3856)	0.40290180725250085
  (0, 3179)	0.3171484171489836
  (0, 2882)	0.1167420637793107
  (0, 2439)	0.3263645438446089
  (0, 286)	0.21068442986350108
  (1, 8987)	0.19076992283327313
  (1, 8451)	0.241347759694005
  (1, 7540)	0.295168601816553
  (1, 7243)	0.46797319025145806
  (1, 6834)	0.2715779250343751
  (1, 5107)	0.2491694815092345
  (1, 5066)	0.2816918272186016
  (1, 2209)	0.295168601816553
  (1, 1791)	0.36655138935443365
  (1, 1684)	0.2917658964829695
  (1, 1543)	0.2574225300294712
  (1, 265)	0.10567061031244322
  (2, 9194)	0.23718517911587875
  :	:
  (2924, 1572)	0.27305471473248605
  (2924, 886)	0.21117289402286005
  (2924, 286)	0.20143116247843124
  (2925, 9218)	0.28388076893325326
  (2925, 8196)	0.30898998753794593
  (2925, 7594)	0.18584597506719103
  (292

In [74]:
#Testing the machine learning model
#Logistic regression

In [75]:
model = LogisticRegression(max_iter=1000)

In [76]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

In [78]:
#Model Evaluation
# Accuracy score

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print(training_data_accuracy)
#It is giving 86% accurate prediction of Y_train means 86% of tweets are predicted accurately according to the Y_train 

0.868681693989071


In [80]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print(test_data_accuracy)

0.7769808743169399


In [81]:
#Saving training model 
import pickle
filename = 'trained_model.sav'
pickle.dump(model,open(filename, 'wb'))